# Simulering av aksjonspotensialet med virrevandring av ioner

Prosjekt 3 i TMA4320 våren 2019 - av Thorvald Ballestad, Jonas Bueie og Herman Sletmoen (gruppe 3)

I denne prosjektoppgaven skal vi simulere aksjonspotensialet i celler, som er raske potensialvariasjoner mellom innsiden og utsiden av celler og som gjerne forplanter seg som et signal langs nerveceller.
Aksjonspotensialet er en konsekvens av ioner som beveger seg inn og ut av celler gjennom cellemembranen i en diffusjonsprosess, påvirket av et elektrisk potensiale.

Vi kommer først til å diskutere noe teori rundt diffusjon og simulere en ren diffusjonsprosess med virrevandring.
Deretter simulerer vi diffusjonsprosessen i et elektrisk potensiale, før vi går løs på å simulere prosessen mellom inn- og utsiden av en cellemembran og til slutt selve aksjonspotensialet.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import constants as consts

# Setup plotting parameters
newparams = {'axes.labelsize': 15,
             'axes.linewidth': 1,
             'lines.linewidth': 1.5, 
             'figure.figsize': (16, 8),
             'ytick.labelsize': 15,
             'xtick.labelsize': 15,
             'ytick.major.pad': 5,
             'xtick.major.pad': 5,
             'legend.fontsize': 15,
             'legend.frameon': True, 
             'legend.handlelength': 1.5,
             'axes.titlesize': 20,
             'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral'}
plt.rcParams.update(newparams)

# TODO: remove before handing in
try:
    heaviside = np.heaviside
except:
    def heaviside(x1, x2):
        return np.where(x1 < 0, 0, np.where(x1 > 0, 1, x2))


## Oppgave 2 - diffusjonslikningen

Diffusjonslikningen i én dimensjon uttrykker at den romlige distribusjonen $\phi = \phi(x, t)$ over posisjonen $x$ av en substans som har diffundert i en tid $t$ oppfyller
$$\frac{\partial \phi(x, t)}{\partial t} = \frac{\partial}{\partial x} \left(D(x) \frac{\partial \phi(x, t)}{\partial x}\right),$$
der $D(x)$ er en generelt posisjonsavhengig diffusjonskoeffisient.

### Oppgave 2.1 - eksponensiell løsning

Når $D(x) = D$ er posisjonsuavhengig, reduseres diffusjonslikningen til 
$$\frac{\partial \phi(x, t)}{\partial t} = D \frac{\partial^2 \phi(x, t)}{\partial x^2},$$
Da er
$$\phi(x, t) = \frac{1}{\sqrt{4 \pi D t}} e^{-\frac{(x - x_0)^2}{4 D t}}$$
en løsning av diffusjonslikningen, siden
$$\frac{\partial \phi(x, t)}{\partial t} = D \frac{\partial^2 \phi(x, t)}{\partial x^2} = \left(\frac{(x-x_0)^2}{4Dt^2} - \frac{1}{2t}\right) \frac{1}{\sqrt{4 \pi D t}} e^{-\frac{(x - x_0)^2}{4 D t}}$$

### Oppgave 2.2.1 - diffusjon fra initiell punktsamling

Vi skal nå å finne distribusjonen $\phi(x, t)$ ved et vilkårlig tidspunkt for en initiell punktsamling i $x_0$
$$\phi(x, 0) = \delta(x - x_0) = \frac{1}{2 \pi} \int_{-\infty}^{+\infty} e^{-i w (x - x_0)} \mathrm{d} w,$$
der $\delta(x)$ er Diracs deltafunksjon.

#### Med taylorutvikling

Vi antar at distribusjonen i en gitt posisjon $x$ kan taylorutvikles om starttidspunktet $t = 0$ som
$$\phi(x, t) = \sum_{n=0}^{\infty} \frac{t^n}{n!} \frac{\partial^n \phi}{\partial t^n}(x, 0).$$
Den tidsderiverte av første orden ved et vilkårlig tidspunkt er gitt direkte ved diffusjonslikningen som
$$\frac{\partial \phi(x, t)}{\partial t} = D \frac{\partial^2 \phi(x, t)}{\partial x^2}.$$
Ved å utnytte diffusjonslikningen, og å bytte derivasjonsrekkefølge, finner vi den tidsderiverte av andre orden
$$\frac{\partial^2 \phi(x, t)}{\partial t^2} = \frac{\partial}{\partial t} D \frac{\partial^2 \phi(x, t)}{\partial x^2} = D \frac{\partial^2}{\partial x^2}\frac{\partial \phi(x, t)}{\partial t} = D^2 \frac{\partial^4 \phi(x, t)}{\partial x^4}.$$
Argumentet kan gjentas flere ganger, og vi får generelt
$$\frac{\partial^n \phi(x, t)}{\partial t^n} = D^n \frac{\partial^{2 n} \phi(x, t)}{\partial x^{2 n}}.$$

Ved $t = 0$ får vi
$$\frac{\partial^n \phi}{\partial t^n}(x, 0) = D^n \frac{\partial^{2 n}}{\partial x^{2 n}} \frac{1}{2 \pi} \int_{-\infty}^{+\infty} e^{-i w (x - x_0)} \mathrm{d} w = D^n \frac{1}{2 \pi} \int_{-\infty}^{+\infty} \frac{\partial^{2 n}}{\partial x^{2 n}} e^{-i w (x - x_0)} \mathrm{d} w = \frac{1}{2 \pi} \int_{-\infty}^{+\infty} (-Dw^2)^{n} e^{-i w (x - x_0)} \mathrm{d} w.$$
Ved innsetting av dette i taylorrekken, får vi at 
$$\phi(x, t) = \frac{1}{2 \pi} \sum_{n=0}^{\infty} \frac{t^n}{n!} \int_{-\infty}^{+\infty} (-Dw^2)^{n} e^{-i w (x - x_0)} \mathrm{d} w = \frac{1}{2 \pi} \int_{-\infty}^{+\infty} \sum_{n=0}^{\infty} \frac{(-D w^2 t)^n}{n!} e^{-i w (x - x_0)} \mathrm{d} w = \frac{1}{2 \pi} \int_{-\infty}^{+\infty} e^{-D w^2 t} e^{-i w (x - x_0)} \mathrm{d} w.$$
der vi har byttet rekkefølge på integrasjonen og summasjonen, og gjenkjent taylorrekken til eksponensialfunksjonen.
Det siste integralet kan skrives som
$$\phi(x, t) = \frac{1}{2 \pi} \int_{-\infty}^{+\infty} e^{-(w \sqrt{D t} + \frac{i(x-x_0)}{2\sqrt{Dt}})^2} e^{-\frac{(x-x_0)^2}{4Dt}} \mathrm{d}w.$$

Om vi innfører $z = w \sqrt{Dt} + \frac{i(x-x_0)}{2\sqrt{Dt}}$, slik at $\mathrm{d}w = \mathrm{d}z/\sqrt{Dt}$, får vi
$$\phi(x, t) = \frac{1}{2 \pi \sqrt{Dt}} e^{-\frac{(x-x_0)^2}{4Dt}} \int_{-\infty}^{+\infty} e^{-z^2} \mathrm{d}z.$$
Integralet i denne likningen har verdien $\sqrt{\pi}$, og vi får til slutt
$$\phi(x, t) = \frac{1}{\sqrt{4 \pi D t}} e^{-\frac{(x - x_0)^2}{4 D t}}.$$

#### Med fourieranalyse (alternativ løsningsmetode)

Med Fourieranalyse kan løsningen finnes med betydelig mindre arbeid ved å innføre en Fouriertransformasjon $\hat{\phi} = \hat{\phi}(w, t)$ av $\phi$ med hensyn kun på posisjonen $x$.
Siden $\widehat{\partial \phi / \partial t} = \partial \hat{\phi} / \partial t$ og $\widehat{\partial^2 \phi / \partial x^2} = (iw)^2 \hat{\phi}$, må fouriertransformasjonen oppfylle
$$\frac{\partial \hat{\phi}(w, t)}{\partial t} = -Dw^2 \hat{\phi}(w, t),$$
hvilket må bety at
$$\hat{\phi}(w, t) = Ae^{-Dw^2t},$$
for en konstant $A$.
Distribusjonen $\phi$ må da være gitt ved den inverse fouriertransformasjonen
$$\phi(x, t) = \frac{A}{\sqrt{2 \pi}} \int_{-\infty}^{+\infty} e^{-Dw^2t}e^{iwx} \mathrm{d}w,$$
som oppfyller initialbetingelsen dersom vi velger $A = 1/\sqrt{2 \pi}$ og gjør koordinatskiftet $x \rightarrow x - x_0$.
Vi kan så finne $\phi$ ved å beregne integralet på samme måte som før, eller simpelthen ved å slå opp den inverse fouriertransformasjonen av $\hat{\phi}$ i en tabell.

### Oppgave 2.2.2 - fysisk tolkning av diffusjonskoeffisienten

Med $\sigma^2 = 2 D t$, er løsningen 
$$\phi = \frac{1}{\sqrt{2 \pi \sigma^2}} e^{-\frac{(x - x_0)^2}{2 \sigma^2}},$$
som tilsvarer sannsynlighetsfordelingen til en normalfordeling med forventningsverdi $x_0$ og varians $\sigma^2$.
Siden variansen vokser lineært med tiden og beskriver spredningen til fordelingen, er diffusjonskoeffisienten $D$ et mål på hvor raskt substansen diffunderer.

### Oppgave 2.2.3 - diffusjon fra vilkårlig initiell spredning 

Diffusjonslikningen er lineær, slik at en lineær kombinasjon av flere løsninger også løser likningen.
Vi kan, ved hjelp av den fundamentale løsningen vi fant i oppgave 2.2.1, utnytte denne egenskapen til å uttrykke distribusjonen ved et vilkårlig tidspunkt $t$, gitt en initiell fordeling $\phi(x, 0) = f(x).$

Dersom
$$\phi(x, t) = \frac{1}{\sqrt{4 \pi D t}} e^{-\frac{(x - x_0)^2}{4 D t}}$$
løser diffusjonslikningen med $\phi(x, 0) = \delta(x - x_0),$ vil lineærkombinasjonen
$$\phi(x, t) = \frac{1}{\sqrt{4 \pi D t}} \sum_{n=-\infty}^{+\infty} c_n e^{-\frac{(x - x_n)^2}{4 D t}}$$
være en løsning med initialbetingelsen $\phi(x, 0) = \sum_{n=-\infty}^{+\infty} c_n \delta(x - x_n),$ og integralet
$$\phi(x, t) = \frac{1}{\sqrt{4 \pi D t}} \int_{-\infty}^{+\infty} f(y) e^{-\frac{(x - y)^2}{4 D t}} \mathrm{d}y$$
vil være en løsning med en generell initiell distribusjon $\phi(x, 0) = \int_{-\infty}^{+\infty} g(y) \delta(x - y) \mathrm{d}y = f(x).$
Summasjonstegnet er enkelt og greit erstattet med et integral, mens $f(y) \mathrm{d}y$ spiller rollen til konstantene $c_n$.

## Oppgave 3 - virrevandring uten potensiale

I denne oppgaven utsetter vi 1000 partikler som først befinner seg samlet i origo for en virrevandringsprosess over 100 tidssteg.
Ved hvert tidssteg hopper hver partikkel enten til høyre eller venstre med like stor sannsynlighet.
Til slutt visualiserer vi posisjonsfordelingen til alle partiklene med et histogram, og sammenlikner fordelingen med en normalkurve tilpasset dataene.

In [ ]:
N = 1000
n_steps = 100

plt.title("Distribution of %d initially gathered particles after %d random walk steps" % (N, n_steps))
plt.xlabel("Position")
plt.ylabel("Proportion of particles")
    
# simulate random walk
pos = np.zeros(N)
steps = np.random.randint(0, 2, size=(n_steps, N))*2 - 1 # Generate random sequence of +1 and -1
step  = np.sum(steps, axis=0)
pos += step
plt.hist(pos, normed = True, label="simulated")

# fit normal curve
mean, stddev = norm.fit(pos)
x = np.linspace(*plt.xlim(), 100)
p = norm.pdf(x, mean, stddev)
plt.plot(x, p, label="theoretical")

plt.legend()
plt.show()

Fra teorien i seksjon 1 i oppgaveteksten vet vi at virrevandring med infinitesimale tids- og posisjonssteg svarer til diffusjon, som beskrevet av diffusjonslikningen.
I tillegg viste vi i oppgave 2 at en diffusjonsprosess med all masse initielt konsentrert i ett punkt etter en tid $t$ er fordelt som en normalfordeling med varianse proporsjonal med $t$ omkring startpunktet.
Vi ser av figuren at den simulerte partikkelfordelingen og den tilpassede normalkurven samsvarer svært godt med hverandre og denne teorien.

## Oppgave 5 - virrevandring i tidsuavhengig potensiale

I denne oppgaven simulerer vi en tilsvarende virrevandringsprosess som i oppgave 3, men her i et potensiale der sannsynligheten for å hoppe til venstre eller høyre fra et punkt avhenger av potensialet rundt punktet.
Ved hjelp av statistisk mekanikk kan man vise at sannsynligheten for at en partikkel hopper til høyre fra et punkt i posisjonen $x$ i en slik virrevandringsprosess er
$$P^+ = \frac{1}{1 + e^{-\beta (V(x-h)-V(x+h))}}$$
der $\beta = 1 / k_B T$ er invers termisk energi og $V(x \mp h)$ er partikkelens potensielle energi i nabopunktene.
Sannsynligheten for å hoppe til venstre er da
$$P^- = 1 - P^+$$
Simuleringen vil gjøres med flere ulike potensialer, men ellers samme parametere og initialbetingelse som i oppgave 3.


In [ ]:
def P_right(x, V, betak = 1):
    ratio = np.exp(-betak * (V(x-1) - V(x+1)))
    return 1/(1 + ratio)

def simulate(pos, V, n_steps = n_steps, betak = 1):
    """Finds pos after n_steps given starting pos and potential V
    Parameters:
     pos    : array of starting position for each particle. dtype=int.
     V      : function V(x) giving potential at point x
     n_steps: number of steps to run
     
    Returns:
     pos    : positions for each particle after    
    """
    # We use pos as index in P_right_pos, so it must have dtype int
    assert pos.dtype == np.int64, "Pos must have type int (int64)"
    
    N = pos.shape[0]
    
    # rand is a n_steps x N-array with random numbers from the interval [0, 1),
    # each row represents a step and each column a particle.
    # We use this number to take a decision given a probability.
    # ie. P(A) = P_A makes us do if (rand < P_a): A else: not A.
    rand = np.random.rand(n_steps, N)
    
    # After n_steps this is the possible interval on which we can find a particle
    x = np.arange(-n_steps, n_steps + 1)
    
    # The probability of going left/right in a given point is only dependent 
    # on the position, not time, so calculate probability of going right in each point
    # for the given potential
    P_right_pos = P_right(x, V, betak)
    
    pos = np.copy(pos) # Do not change the input array
    
    for t in range(n_steps): # Go through time
        
        # Prob of going right for each particle
        # hint: numpy indexing is super fancy, so you can do
        # i = np.array([3,3,8,2])
        # A = np.arange(100)
        # A[i] -> will return an array with elements 3, 3, 8, 2 from A!!!
        # Add x[0] to index so that pos = -n_steps is shifted to pos = 0.
        P = P_right_pos[pos - x[0]]
        
        # Decide to go either right or left
        step = np.where(rand[t] <= P, 1, -1)
        pos += step
    
    return pos

class Tester:
    """Helper class helping explore solutions by implementing the most common tasks
    Attempts to be so non-intrusive and flexible as possible"""
    
    def __init__(self, V_function, k_beta_list, n_steps = 100, N = 1000,
                 subplots = True, horizontal = False):
        # Simulation-relevant variables
        self.V           = V_function
        self.k_beta_list = k_beta_list
        self.n_steps     = n_steps
        self.N           = N
        
        # Meta/plotting variables
        self.subplots    = subplots # Plot each k_beta-value in own subfig
        self.n           = len(k_beta_list)
        self.horizontal  = horizontal # Should subplots be horizontal?
        self.has_simulated = False # Has the simulation been run?
        
        self.pos = np.zeros((self.n, N), dtype = int) # Create one row for each k_beta-value
        
    def simulate(self):
        """Simulate for each value of k_beta"""
        
        if self.has_simulated:
            print("Warn: Simulation has already been run before!")
            
        for i in range(self.pos.shape[0]):
            self.pos[i] = simulate(self.pos[i], self.V, self.n_steps, self.k_beta_list[i])
            
        self.has_simulated = True
    
    def _gen_subplot_string(self, i):
        grid = "1" + str(self.n) if self.horizontal else str(self.n) + "1" # 1xn or nx1 grid. This could probably have been written prettier, using reverse or something.
        return grid + str(i)
        
    def plot(self):
        for i, p in enumerate(self.pos):
            if self.subplots:
                plt.subplot(self._gen_subplot_string(i))
                plt.title("$k \\beta:$ %r" % self.k_beta_list[i])
            #TODO: bins-calculation could be more sophisticated
            plt.hist(p, bins=100)
            
        plt.tight_layout()
        
    def plot_potential(self, V_scale_factor = 1, V_shift = 0):
        x = np.linspace(np.amin(self.pos), np.amax(self.pos), 100)
        if self.subplots:
            for i in range(self.n):
                plt.subplot(self._gen_subplot_string(i))
                xmin, xmax = plt.xlim() # Do not change xlim when plotting V
                plt.plot(x, self.V(x) * V_scale_factor + V_shift)
                plt.xlim(xmin, xmax)
        else:
            plt.plot(x, self.V(x) * V_scale_factor + V_shift)

### Oppgave 5.1 - lineært potensiale

Først simulerer vi virrevandringen i det lineære potensialet
$$V(x) = k x$$
for en konstant $k$, som representerer en konstant kraft på partiklene.

In [ ]:
def V_linear(x):
    return x

test = Tester(V_linear, [0.1, 1, 10], N = 1000, n_steps = 200, horizontal = True)
test.plot()
plt.show() # We do not want plt.show() inside Teseter, in order to get more flexibility (for example in case we want multiple plots within same fig)
test.simulate()
test.plot()
test.plot_potential()
plt.show()

In [ ]:
lin_pot_tester = Tester(V_linear, [0.1, 1, 10], N = 1000, n_steps = 100, subplots = False)
lin_pot_tester.simulate()
lin_pot_tester.plot()
lin_pot_tester.plot_potential()
plt.show()

### Oppgave 5.2 - bokspotensiale

Vi simulerer nå virrevandringsprosessen i et bokspotensiale
$$V(x) = \begin{cases} k & -3h < x < +3h \\ 0 & \text{ellers} \\ \end{cases}$$
som representerer en partikkel som kan bevege seg uten påvirkning av krefter i flere regioner der potensialet er flatt, og som utsettes for en kraft i overgangsområdene mellom disse flate regionene.

In [ ]:
def V_membrane(x, d = 3):
    return np.where(np.logical_and(-d < x, x < d), 1, 0)

mem_pot_tester = Tester(V_membrane, [0.1, 1, 10], N = 10000, n_steps = 100, subplots = False)
mem_pot_tester.simulate()
mem_pot_tester.plot()
mem_pot_tester.plot_potential(V_scale_factor = 50)
plt.show()

### Oppgave 5.3 - lineært potensiale med flate endestykker

Her simulerer vi virrevandringsprosessen i potensialet
$$V(x) = \begin{cases} -k & x \leq -3h \\ k \left(-1 + 2 \frac{x+3h}{6h} \right) & -3h < x < +3h \\ +k & x \geq +3h \\ \end{cases}$$
som representerer en partikkel som påvirkes av en konstant kraft i et avgrenset område, men som ellers er upåvirket.

In [ ]:
def V_linear_edge(x):
    # Alternatively reuse V_membrane here
    return np.where(np.logical_and(-3 < x, x < 3),\
                                   -1 + 2*(x + 3)/6,\
                                    np.sign(x))

edge_pot_tester = Tester(V_linear_edge, [0.1, 1, 10], N = 10000, n_steps = 100, subplots = False)
edge_pot_tester.simulate()
edge_pot_tester.plot()
edge_pot_tester.plot_potential(V_scale_factor = 50)
plt.show()

## Oppgave 7 og 8 - virrevandring av ioner i cellemembran med tidsavhengig potensiale og simulering av aksjonspotensialet

In [ ]:
def V_diff(pos, wall_pos = 0):
    """Find voltage difference across membrane caused by charge difference
    Params:
     wall_pos : position of membrane wall. Inside of cell is right of this position.
    """
    C = 10 # Capacitance of cell. From task description
    Q_diff = len(pos[pos > wall_pos]) - len(pos)/2 # Charge difference across membrane
    return Q_diff / C

def V_membrane_charge(pos, membrane_constant, wall_pos = 0):
    return membrane_constant * V_membrane(pos, d = 1) + np.where(pos > wall_pos, V_diff(pos, wall_pos), 0)

def V_membrane_capacitance(x, pos, pos_secondary, V0):
    """Returns an energy potential array for each x, in J
    pos is pos
    pos_secondary is the pos of the other ions, used to find charge diff.
    V0 is V0"""
    # Assumes a ton of variables exists
    membrane = V0*V_membrane(x, d=1)
    v_diff = V_diff(pos, pos_secondary)
    
    return beta*(membrane\
         + v_diff * consts.e * heaviside(x, 0.5))

def simulate_v2(V0_Na_beta=1, V0_K_beta=1, n_steps=1000, L=50, action_potential=False, pump=False):
    ## Let's set us some constants and values
    # Firstly, we assume the inside of the cell is
    # to the right, outside to the left.

    ## System related ##
    C = 0.07 # eM/V, capacitance.
    C *= 1e3 # Convert from M to mM
    T = 273 + 37 # K
    beta = 1 / (consts.k*T)
    
    V0_Na = V0_Na_beta / beta
    V0_K = V0_K_beta / beta

    # Set number inside and outside, then calculate total
    N_Na_inside = 50
    N_Na_outside = 1450
    N_K_inside = 1400
    N_K_outside = 50
    N_Na = N_Na_inside + N_Na_outside
    N_K = N_K_inside + N_K_outside

    # Outside concentrations constant throughout the entire sim.
    outside_Na_concentration = 145 # mM
    outside_K_concentration = 5 #mM
    
    # Voltage limits for the action potential (Na- and K-gates)
    V_beta_closed = 50 # Same for K and Na
    V_closed = V_beta_closed / beta
    lower_v_limit = -70e-3 # Volts
    upper_v_limit = 30e-3 # Volts
    
    # Pump:
    num_Na_pump_transport = 3
    num_K_pump_transport = 2
    pump_period = 10 # Number of time steps between each pumping
    
      
    if action_potential:
        V_Na = V_closed
        V_K = V_closed
    else:
        V_Na = V0_Na
        V_K = V0_K

    cons_scaling_factor = 0.1 # mM, the concentration each 'particle'
                              # in the simulation represents

    ## End of constants and values ##
    def V_diff(pos, pos_secondary):
        """Returns voltage difference, in volts"""
        inside_concentration = (len(pos[pos > 0])\
                             + len(pos_secondary[pos_secondary > 0]))\
                             * cons_scaling_factor
        concentration_diff = inside_concentration\
                           - outside_Na_concentration\
                           - outside_K_concentration
        return concentration_diff/C

    # Create the random values
    rand_Na = np.random.rand(n_steps, N_Na)
    rand_K = np.random.rand(n_steps, N_K)

    # Put the ions at their positions
    pos_Na = np.append(-L//4*np.ones(N_Na_outside, dtype=int),
                        L//4*np.ones(N_Na_inside, dtype=int))
    pos_K = np.append(-L//4*np.ones(N_K_outside, dtype=int),
                       L//4*np.ones(N_K_inside, dtype=int))

    # After n_steps this is the possible interval on which we can find a particle
    x = np.arange(-L//2, L//2 + 1)

    v_diffs = [] # V_diff is appended inside V_membrane_capacitance

    for t in range(n_steps): # Go through time
        v_diff = V_diff(pos_Na, pos_K)
        
        if action_potential:
            if v_diff < lower_v_limit:
                V_Na = V0_Na
                V_K = V_closed
            if v_diff > upper_v_limit:
                V_Na = V_closed
                V_K = V0_K
            
        # Prob of going right at a given point in space   
        # We do not use P_right() from earlier on purpose
        v_diff_array = v_diff * heaviside(x, 0.5) * consts.e
        membrane = V_membrane(x, d=1)
        
        v_total_Na = beta*(v_diff_array + V_Na*membrane)
        v_total_K = beta*(v_diff_array + V_K*membrane)
        
        ratio_Na = np.exp(np.roll(v_total_Na, -1) - np.roll(v_total_Na, 1))
        ratio_K  = np.exp(np.roll(v_total_K, -1) - np.roll(v_total_K, 1))
        
        P_right_pos_Na = 1 / (1+ratio_Na)
        P_right_pos_K = 1 / (1+ratio_K)
        
        v_diffs.append(v_diff * consts.e * beta)

        # Add box walls (indecies)
        right_wall = -1
        left_wall = 0
        P_right_pos_Na[right_wall] = 0
        P_right_pos_Na[left_wall] = 1
        P_right_pos_K[right_wall] = 0
        P_right_pos_K[left_wall] = 1

        # Prob of going right for a given particle
        P_Na = P_right_pos_Na[pos_Na - x[0]]
        P_K  = P_right_pos_K[pos_K - x[0]]

        # Decide to go either right or left
        step_Na = np.where(rand_Na[t] <= P_Na, 1, -1)
        step_K  = np.where(rand_K[t] <= P_K , 1, -1)

        pos_Na += step_Na
        pos_K  += step_K
        
        # TODO pump: If there are no ions on the correct position,
        #            should it wait 10 more steps or do it as soon as possible?
        if pump:
            if t % pump_period:
                continue
            pos_Na.sort() # TODO: How is sort() implemented? Does it cost much or not, if so, how can we do it differently?
            pos_K.sort()
            index_Na = np.argwhere(pos_Na > 0)[:num_Na_pump_transport+1]
            index_K = np.argwhere(pos_K < 0)[:num_K_pump_transport+1]
            pos_Na[index_Na] = -1
            pos_K[index_K] = 1
        
    return pos_Na, pos_K, v_diffs

### Oppgave 7.1 og 7.2 - virrevandring i cellemembran med identiske og ulike ionekanalpotensialer

In [ ]:
ONE_FIG = True
for V0_Na, V0_K in [(1,1), (10,1), (1,10)]:
    pos_Na, pos_K, v_diffs = simulate_v2(V0_Na, V0_K)
    label = "$\\beta V_{Na}: " + str(V0_Na) + ",\\quad \\beta V_{K}: " + str(V0_K) + "$"
    plt.plot(v_diffs, label=label)
    if ONE_FIG:
        plt.legend()
    else:
        plt.title("$\\beta V_{Na}: " + str(V0_Na) + ",\\quad \\beta V_{K}: " + str(V0_K) + "$")
        plt.show()
plt.show()

### Oppgave 8.1 - aksjonspotensialet uten ionepumpe

In [ ]:
pos_Na, pos_K, v_diffs = simulate_v2(action_potential=True, pump=False)
plt.plot(v_diffs)
plt.show()

### Oppgave 8.2 - aksjonspotensialet med ionepumpe

In [ ]:
pos_Na, pos_K, v_diffs = simulate_v2(action_potential=True, pump=True)
plt.plot(v_diffs)
plt.show()